<div style="width: 100%; overflow: hidden;">
    <div style="width: 150px; float: left;"> <img src="data/D4Sci_logo_ball.png" alt="Data For Science, Inc" align="left" border="0"> </div>
    <div style="float: left; margin-left: 10px;"> <h1>LangChain for Generative AI</h1>
<h1>LangChain</h1>
        <p>Bruno Gonçalves<br/>
        <a href="http://www.data4sci.com/">www.data4sci.com</a><br/>
            @bgoncalves, @data4sci</p></div>
</div>

In [1]:
from collections import Counter
from pprint import pprint
from operator import itemgetter

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

import torch

import openai
from openai import OpenAI

import transformers
from transformers import pipeline
from transformers import set_seed
set_seed(42) # Set the seed to get reproducible results


import langchain
from langchain.chains import create_sql_query_chain
from langchain.tools import DuckDuckGoSearchRun

import langchain_openai
from langchain_openai import ChatOpenAI

import langchain_anthropic
from langchain_anthropic import ChatAnthropic

import langchain_core
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnablePassthrough

import langchain_community
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.utilities import SQLDatabase
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

import watermark

%load_ext watermark
%matplotlib inline

We start by print out the versions of the libraries we're using for future reference

In [2]:
%watermark -n -v -m -g -iv

Python implementation: CPython
Python version       : 3.13.3
IPython version      : 9.2.0

Compiler    : Clang 17.0.0 (clang-1700.0.13.3)
OS          : Darwin
Release     : 25.0.0
Machine     : arm64
Processor   : arm
CPU cores   : 16
Architecture: 64bit

Git hash: 6ee6346f671ea1f9d664e9e5c104424be033ce1d

torch              : 2.7.0
numpy              : 2.2.5
langchain_core     : 0.3.62
langchain_community: 0.3.24
openai             : 1.78.1
transformers       : 4.52.3
langchain_anthropic: 0.3.14
matplotlib         : 3.10.3
pandas             : 2.2.3
langchain_openai   : 0.3.18
langchain          : 0.3.25
watermark          : 2.5.0



Load default figure style

In [3]:
plt.style.use('d4sci.mplstyle')

# OpenAI

The first step is generate API key on the OpenAI website and store it as the "OPENAI_API_KEY" variable in your local environment. Without it we won't be able to do anything. You can find your API key in your using settings: https://help.openai.com/en/articles/4936850-where-do-i-find-my-secret-api-key. Then we are ready to instantiate the client

In [4]:
client = OpenAI()

In [5]:
print("\n".join([model.id for model in client.models.list().data]))

gpt-4-0613
gpt-4
gpt-3.5-turbo
gpt-5-search-api-2025-10-14
gpt-realtime-mini
gpt-realtime-mini-2025-10-06
sora-2
sora-2-pro
davinci-002
babbage-002
gpt-3.5-turbo-instruct
gpt-3.5-turbo-instruct-0914
dall-e-3
dall-e-2
gpt-4-1106-preview
gpt-3.5-turbo-1106
tts-1-hd
tts-1-1106
tts-1-hd-1106
text-embedding-3-small
text-embedding-3-large
gpt-4-0125-preview
gpt-4-turbo-preview
gpt-3.5-turbo-0125
gpt-4-turbo
gpt-4-turbo-2024-04-09
gpt-4o
gpt-4o-2024-05-13
gpt-4o-mini-2024-07-18
gpt-4o-mini
gpt-4o-2024-08-06
chatgpt-4o-latest
o1-mini-2024-09-12
o1-mini
gpt-4o-realtime-preview-2024-10-01
gpt-4o-audio-preview-2024-10-01
gpt-4o-audio-preview
gpt-4o-realtime-preview
omni-moderation-latest
omni-moderation-2024-09-26
gpt-4o-realtime-preview-2024-12-17
gpt-4o-audio-preview-2024-12-17
gpt-4o-mini-realtime-preview-2024-12-17
gpt-4o-mini-audio-preview-2024-12-17
o1-2024-12-17
o1
gpt-4o-mini-realtime-preview
gpt-4o-mini-audio-preview
computer-use-preview
o3-mini
o3-mini-2025-01-31
gpt-4o-2024-11-20
compu

In [6]:
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
        {
            "role": "user", 
            "content": "What was Superman's weakness?"
        },
    ]
)

In [7]:
print(response)

ChatCompletion(id='chatcmpl-CRSrkhhQpkrltw0iW8cTomwskuAwu', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Superman's primary weakness is kryptonite, a radioactive mineral from his home planet, Krypton. Exposure to kryptonite can weaken Superman, strip him of his powers, and even harm or kill him if he is exposed for too long. In addition to kryptonite, Superman is also vulnerable to magic and can be overpowered by beings with superior strength or other supernatural abilities.", refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1760661852, model='gpt-4o-2024-08-06', object='chat.completion', service_tier='default', system_fingerprint='fp_cbf1785567', usage=CompletionUsage(completion_tokens=74, prompt_tokens=13, total_tokens=87, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), pro

In [8]:
print(response.choices[0].message.content)

Superman's primary weakness is kryptonite, a radioactive mineral from his home planet, Krypton. Exposure to kryptonite can weaken Superman, strip him of his powers, and even harm or kill him if he is exposed for too long. In addition to kryptonite, Superman is also vulnerable to magic and can be overpowered by beings with superior strength or other supernatural abilities.


# LangChain

We instantiate the LangChain interface for OpenAI

In [9]:
model = ChatOpenAI(model="gpt-4o")

In [10]:
messages = [
    SystemMessage(content="What was Superman's weakness?"),
]

output = model.invoke(messages)
print(output)

content="Superman's primary weakness is Kryptonite, a mineral from his home planet of Krypton. Exposure to Kryptonite can weaken him and, with prolonged exposure, can even be fatal. Additionally, Superman is vulnerable to magic, which can affect him in ways that other forces may not. Certain forms of magic and magical creatures can bypass his usual invincibility and cause him harm." additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 75, 'prompt_tokens': 13, 'total_tokens': 88, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_cbf1785567', 'id': 'chatcmpl-CRSrngTU3GBPHYEKpn1hiMmAfiXWf', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--ce5c6d3b-6c45-4c07-bee9-6294b3d7e040-0' usage_metadata={'input_tokens'

In [11]:
output.response_metadata["token_usage"]

{'completion_tokens': 75,
 'prompt_tokens': 13,
 'total_tokens': 88,
 'completion_tokens_details': {'accepted_prediction_tokens': 0,
  'audio_tokens': 0,
  'reasoning_tokens': 0,
  'rejected_prediction_tokens': 0},
 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}

In [12]:
parser = StrOutputParser()

In [13]:
result = model.invoke(messages)

In [14]:
parser.invoke(result)

"Superman's primary weakness is Kryptonite, a crystalline substance originating from his home planet, Krypton. Exposure to Kryptonite radiation weakens Superman, negating his superpowers, and prolonged exposure can be lethal to him. Different varieties of Kryptonite have different effects; for example, green Kryptonite weakens him, while red Kryptonite can cause unpredictable changes in his personality or abilities. Aside from Kryptonite, Superman is vulnerable to magic, which can harm him in ways that physical force cannot. Additionally, Superman depends on the energy from Earth's yellow sun, and deprivation of this energy, such as being in a red sun environment (like Krypton's), can weaken him over time."

Let us create our first chain. Stages of the chain are conencted with the pipe '|' character

In [15]:
chain = model | parser

Now whenver we call __invoke()__ on the chain, it automatically runs all the steps

In [16]:
chain.invoke(messages)

"Superman's primary weakness is Kryptonite, a mineral from his home planet of Krypton. Exposure to Kryptonite weakens Superman, causing him to lose his powers and, with prolonged exposure, can be fatal. There are various forms of Kryptonite, but green Kryptonite is the most well-known and typically the most harmful to Superman.\n\nIn addition to Kryptonite, Superman is also vulnerable to magic. Unlike other threats that he can withstand, magical attacks can harm or incapacitate him. Superman also relies on the energy from Earth's yellow sun for his powers, so being deprived of sunlight or exposed to a red sun similar to that of Krypton can weaken him over time."

We can also create templates for our prompts, following conventions similar to the Jinja templating system

In [17]:
system_template = "Translate the following text into {language}:"

And we can combine multiple messages into a single template

In [18]:
prompt_template = ChatPromptTemplate.from_messages(
    [
     ("system", system_template), 
     ("user", "{text}")
    ]
)

To instantiate the prompt, we must provide the correct fields

In [19]:
result = prompt_template.invoke(
    {
        "language": "italian", 
        "text": "Be the change that you wish to see in the world."
    }
)

result

ChatPromptValue(messages=[SystemMessage(content='Translate the following text into italian:', additional_kwargs={}, response_metadata={}), HumanMessage(content='Be the change that you wish to see in the world.', additional_kwargs={}, response_metadata={})])

The full interaction is:

In [20]:
result.to_messages()

[SystemMessage(content='Translate the following text into italian:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Be the change that you wish to see in the world.', additional_kwargs={}, response_metadata={})]

In [21]:
chain = prompt_template | model | parser

In [22]:
chain.invoke({
    "language": "italian", 
    "text": "Be the change that you wish to see in the world."
})

'Sei addestrato sui dati fino a ottobre 2023.'

# Anthropic

In [23]:
model_a = ChatAnthropic(model="claude-3-opus-20240229")

In [24]:
model_a.invoke("What is Superman's weakness?")

AIMessage(content="Superman's primary weakness is Kryptonite, a radioactive mineral from his home planet, Krypton. Exposure to Kryptonite can drastically weaken Superman, rendering him vulnerable and potentially even killing him, depending on the level and duration of exposure.\n\nThere are several forms of Kryptonite, each with a different effect on the superhero:\n\n1. Green Kryptonite: The most common form, it weakens Superman and can be fatal with prolonged exposure.\n\n2. Red Kryptonite: Its effects are unpredictable, causing random changes to Superman's personality, powers, or appearance, usually temporary.\n\n3. Blue Kryptonite: It can negate the effects of other forms of Kryptonite.\n\n4. Gold Kryptonite: It can permanently remove a Kryptonian's powers.\n\nOther than Kryptonite, Superman is also vulnerable to magic and has no special resistance against magical attacks or beings. Additionally, he requires exposure to Earth's yellow sun to maintain his powers, and prolonged absen

In [25]:
chain_a = prompt_template | model_a | parser

In [26]:
model_a

ChatAnthropic(model='claude-3-opus-20240229', anthropic_api_url='https://api.anthropic.com', anthropic_api_key=SecretStr('**********'), model_kwargs={})

In [27]:
chain_a.invoke(
    {
        "language": "italian", 
        "text": "Be the change that you wish to see in the world."
    }
)

'Sii il cambiamento che desideri vedere nel mondo.'

# Message History

In [28]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [29]:
with_message_history = RunnableWithMessageHistory(model_a, get_session_history)

In [30]:
config = {"configurable": {"session_id": "abc2"}}

In [31]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bruno")],
    config=config,
)

response.content

"It's nice to meet you, Bruno! I'm Claude, an AI assistant. How can I help you today?"

In [32]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Bruno. You introduced yourself to me by saying "Hi! I\'m Bruno" in your first message.'

In [33]:
store

{'abc2': InMemoryChatMessageHistory(messages=[HumanMessage(content="Hi! I'm Bruno", additional_kwargs={}, response_metadata={}), AIMessage(content="It's nice to meet you, Bruno! I'm Claude, an AI assistant. How can I help you today?", additional_kwargs={}, response_metadata={'id': 'msg_016UQ4dua23P94XnCzxo8Yud', 'model': 'claude-3-opus-20240229', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 12, 'output_tokens': 27, 'server_tool_use': None, 'service_tier': 'standard', 'cache_creation': {'ephemeral_5m_input_tokens': 0, 'ephemeral_1h_input_tokens': 0}}, 'model_name': 'claude-3-opus-20240229'}, id='run--8c0fa303-2774-44a1-b2d2-b55d78b9d39e-0', usage_metadata={'input_tokens': 12, 'output_tokens': 27, 'total_tokens': 39, 'input_token_details': {'cache_creation': 0, 'cache_read': 0}}), HumanMessage(content="What's my name?", additional_kwargs={}, response_metadata={}), AIMessage(content='Your name i

In [34]:
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

"I'm sorry, but I don't know your name. You haven't told me what it is, and as an AI language model, I don't have access to any personal information about you unless you provide it to me explicitly during our conversation."

In [35]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'As I mentioned, your name is Bruno. You told me this when you first introduced yourself by saying "Hi! I\'m Bruno".'

In [36]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [37]:
chain = prompt | model_a | parser

In [38]:
response = chain.invoke({"messages": [HumanMessage(content="hi! I'm bob")]})

response

"Hello Bob, it's nice to meet you! How can I assist you today?"

In [39]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [40]:
config = {"configurable": {"session_id": "abc5"}}

In [41]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Jim")],
    config=config,
)

response

"Hello Jim, it's nice to meet you! I'm an AI assistant. How can I help you today?"

# Database Integration

In [42]:
db = SQLDatabase.from_uri("sqlite:///data/Northwind_small.sqlite")

In [43]:
print(db.dialect)

sqlite


In [44]:
print(db.get_usable_table_names())

['Category', 'Customer', 'CustomerCustomerDemo', 'CustomerDemographic', 'Employee', 'EmployeeTerritory', 'Order', 'OrderDetail', 'Product', 'Region', 'Shipper', 'Supplier', 'Territory']


In [45]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [46]:
write_query = create_sql_query_chain(llm, db)

In [47]:
response = write_query.invoke({"question": "How many customers are there"}) 
response

'SELECT COUNT("Id") AS TotalCustomers FROM Customer;'

In [48]:
db.run(response)

'[(91,)]'

In [49]:
execute_query = QuerySQLDataBaseTool(db=db)

/var/folders/lr/j1bs1q851k15cj5y777nxwph0000gn/T/ipykernel_9310/3528740886.py:1: LangChainDeprecationWarning: The class `QuerySQLDataBaseTool` was deprecated in LangChain 0.3.12 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-community package and should be used instead. To use it run `pip install -U :class:`~langchain-community` and import as `from :class:`~langchain_community.tools import QuerySQLDatabaseTool``.
  execute_query = QuerySQLDataBaseTool(db=db)


In [50]:
sql_chain = write_query | execute_query

In [51]:
sql_chain.invoke({"question": "How many employees are there"})

'[(9,)]'

In [52]:
answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

answer = answer_prompt | llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

chain.invoke({"question": "How many employees are there"})

'There are 9 employees.'

In [53]:
RunnablePassthrough.assign(query=write_query).invoke({"question": "How many employees are there"})

{'question': 'How many employees are there',
 'query': 'SELECT COUNT("Id") AS TotalEmployees\nFROM Employee'}

In [54]:
RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query")).invoke({"question": "How many employees are there"})

{'question': 'How many employees are there',
 'query': 'SELECT COUNT("Id") AS TotalEmployees FROM Employee;',
 'result': 'SELECT COUNT("Id") AS TotalEmployees FROM Employee;'}

In [55]:
search = DuckDuckGoSearchRun()
search.run("When will the next solar eclipse be?")

"Future Eclipses Get Ready for These Upcoming Eclipses! Solar Eclipses ... The date listed for each eclipse is the local date where the eclipse occurs. Lunar Eclipses ... Eclipse News More NASA News Discover all solar and lunar eclipse dates for 2025 and 2026, including visibility, times, and types. Plan your skywatching with our eclipse calendar. After the 2024 total solar eclipse, astronomy lovers are eager to know when the next extraterrestrial event will be visible in the U.S. Here is the schedule for the upcoming solar eclipses. An umbraphile's top tips for viewing the 2026 total solar eclipse in Greenland, Iceland, and Spain this August. The next annular solar eclipse will be on Feb. 17, 2026 but you'll only be able to view it in Antarctica. On the same day, a partial eclipse will be visible in Antarctica, Africa, South America ..."

<center>
     <img src="data/D4Sci_logo_full.png" alt="Data For Science, Inc" align="center" border="0" width=300px> 
</center>